# Parkinson's Freezing of Gait Prediction

Additional useful references:
* Competition homepage: https://www.kaggle.com/competitions/tlvmc-parkinsons-freezing-gait-prediction/overview
* Google Brain Ventilator Pressure Prediction Winning team's solution https://www.kaggle.com/competitions/ventilator-pressure-prediction/discussion/285965
* Feature selection (not used in this notebook but was used for other prototypes): https://www.kaggle.com/code/averkovanika/parkinson-s-fog-mi-based-feature-selection

Loading the libraries + global variables:

In [ ]:
import datetime
import glob
import json
import multiprocessing
import os
import time

import numpy as np
import pandas as pd
import torch
from models import MultiResidualBiGRU
from preprocessing import (
    SeqFoGDataset,
    add_noactivity_lbls,
    convert_g_to_ms2,
    preprocess_defog,
    preprocess_tdcsfog,
)
from pytorch_ranger import Ranger
from scipy.interpolate import interp1d
from sklearn.preprocessing import StandardScaler
from torch import nn, optim
from torch.cuda.amp import autocast
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, SubsetRandomSampler
from tqdm.notebook import tqdm
from training import get_model_id, train
from util import set_seed

RND_SEED = 0
GPU_ID = 0
USE_GPU = True

if torch.cuda.is_available() and USE_GPU:
    gpu_name = torch.cuda.get_device_name(GPU_ID)
    print(f"Using GPU {GPU_ID} - {gpu_name}")
    device = torch.device(f"cuda:{GPU_ID}")
else:
    print(f"CUDA is not available. Using CPU")
    device = torch.device("cpu")

N_CPU_CORES = multiprocessing.cpu_count()

BASE_FOLDER = os.path.join("input")

print(f"Number of CPU cores available: {N_CPU_CORES}")

## Training

In the following piece of code, the parameters correspond to the ones used to obtain the model that achieved the best private score on the Kaggle leaderboard. To train the simplified version of the model detailed in the model summary, just change the "NL" parameter to 1. 

Note: a UserWarning related to the Ranger optimizer should appear during the training, it can be ignored. 

In [ ]:
set_seed(RND_SEED)

PARAMS = {
    "SEED": RND_SEED,  # random seed, for reproducibility
    "DSPART": 1.0,  # eg 0.5 = only consider 50% of the full dataset
    "TRAINPART": 0.8,  # eg 0.8 means 80% = training set / 20% = validation set
    "DOWNHZ": 50,  # by default (None), defog 100Hz sequences are upsampled to
    # 128Hz. Else this value indicates the frequency at which all the sequences
    # are downsampled (in Hz)
    "MAXSEQLEN": 200000,  # maximum sequence length ; None if no limit.
    "MAXCHUNKSIZE": 500000,  # given the important size of some sequences, we
    # give the ability to process them in chunks, and this is the maximum size
    # for 1 chunk
    "BS": 1,  # batch size
    "NWORK": 0,  # number of workers used in dataloaders
    "ISIZE": 3,  # GRU's input size
    "HSIZE": 128,  # GRU's hidden size
    "NL": 3,  # number of layers
    "NC": 4,  # number of classes
    "NEPOCHS": 20,  # number of epochs
    "LR": 1e-3,  # learning rate
    # "WDECAY": 1e-2,  # weight decay
    # "PATIENCE": 3,  # patience in epochs before the lr is modified
    # "FACTOR": 0.5,  # factor by which the lr is multiplied when decreased
    "COSSTART": 0.75,  # determines at what % of the epochs the cosine annealing
    # starts
    "VINTER": 1,  # validation interval in epochs (1 = validation at each epoch)
}

# defining the model, optimizer and criterion
model = MultiResidualBiGRU(
    PARAMS["ISIZE"],
    PARAMS["HSIZE"],
    PARAMS["NC"],
    PARAMS["NL"],
)
model = model.to(device)

MODELS_FOLDER = "models"
model_id = get_model_id(model, extra_info="")
model_path = os.path.join(MODELS_FOLDER, model_id)
params_path = os.path.join(model_path, "params.json")
if not os.path.isdir(model_path):
    os.makedirs(model_path)
with open(params_path, "w", encoding="utf-8") as f:
    json.dump(PARAMS, f, ensure_ascii=False, indent=4)

# loading and preprocessing data
defog_TRAIN_PATH = os.path.join(BASE_FOLDER, "train", "defog")
tDCS_TRAIN_PATH = os.path.join(BASE_FOLDER, "train", "tdcsfog")
train_ds = SeqFoGDataset(
    defog_TRAIN_PATH,
    tDCS_TRAIN_PATH,
    ds_part=PARAMS["DSPART"],
    max_seq_len=PARAMS["MAXSEQLEN"],
    down_hz=PARAMS["DOWNHZ"],
)

# creating training and validation samplers for the associated dataloaders
train_size = int(PARAMS["TRAINPART"] * len(train_ds))
valid_size = len(train_ds) - train_size
indices = torch.randperm(len(train_ds))
train_sampler = SubsetRandomSampler(indices[:train_size])
valid_sampler = SubsetRandomSampler(
    indices[train_size : train_size + valid_size]
)

# training and validation dataloaders
train_loader = DataLoader(
    train_ds,
    batch_size=PARAMS["BS"],
    sampler=train_sampler,
    pin_memory=True,
    num_workers=PARAMS["NWORK"],
)
valid_loader = DataLoader(
    train_ds,
    batch_size=PARAMS["BS"],
    sampler=valid_sampler,
    pin_memory=True,
    num_workers=PARAMS["NWORK"],
)

criterion = nn.CrossEntropyLoss()

opt = Ranger(model.parameters(), lr=PARAMS["LR"])

cos_epoch = int(PARAMS["NEPOCHS"] * PARAMS["COSSTART"])
T_max = (PARAMS["NEPOCHS"] - cos_epoch) * len(train_loader)
scheduler = lr_scheduler.CosineAnnealingLR(opt, T_max)

print(model)

# training loop
history = train(
    train_loader,
    valid_loader,
    model,
    PARAMS["MAXCHUNKSIZE"],
    PARAMS["NEPOCHS"],
    PARAMS["VINTER"],
    criterion,
    opt,
    scheduler,
    cos_epoch,
    device,
    model_path,
    verbose=True,
)

## Loading a model & Inference

In [ ]:
# some auxiliary functions
def read_seq(fpath):
    seq_id = fpath.split(os.path.sep)[-1].split(".")[0]
    seq = pd.read_csv(fpath)
    return seq_id, seq


def resample_seq_df(df, in_hz, out_hz, with_classes=True, with_bool_cols=True):
    in_ms = (1 / in_hz) * 1000
    out_ms = (1 / out_hz) * 1000
    FLOAT_COLS = ["AccV", "AccML", "AccAP"]
    if with_classes:
        CLASSES_COLS = ["StartHesitation", "Turn", "Walking"]
    if with_bool_cols:
        BOOL_COLS = ["Valid", "Task"]

    df["Time"] = pd.to_timedelta(df["Time"] * in_ms, unit="ms")
    df = df.set_index("Time")

    resampled_df = (
        df[FLOAT_COLS]
        .resample(f"{out_ms}ms")
        .mean()  # new val = "mean" in the 7.8125ms interval
        .interpolate()  # sometimes there is no previous value in the 7.8125ms
        # interval: we interpolate (linearly by default)
    )

    cols = []
    if with_classes:
        cols = cols + CLASSES_COLS
    if with_bool_cols:
        cols = cols + BOOL_COLS
    if cols != []:
        resampled_df[cols] = (
            df[cols]
            .resample(f"{out_ms}ms")
            .first()
            .ffill()  # new val = previous val
        )

    # needed as the introduction of NaNs forced pd to make all cols float
    if with_classes:
        resampled_df[CLASSES_COLS] = resampled_df[CLASSES_COLS].astype(int)
    if with_bool_cols:
        resampled_df[BOOL_COLS] = resampled_df[BOOL_COLS].astype(bool)

    return resampled_df


def convert_g_to_ms2(df):
    # 1g = 9.80665m/s^2
    df["AccV"] = df["AccV"] * 9.80665
    df["AccML"] = df["AccML"] * 9.80665
    df["AccAP"] = df["AccAP"] * 9.80665
    return df


def normalize(seq_features):
    return StandardScaler().fit_transform(seq_features)


def preprocess_tdcs_seq(seq_df, device, down_hz=None):
    FEATURES = ["AccV", "AccML", "AccAP"]

    if down_hz is not None:
        # downsample the data from 128Hz to ??Hz
        seq_df = resample_seq_df(
            seq_df, 128, down_hz, with_classes=False, with_bool_cols=False
        )

    # extracting the features columns and normalizing them
    seq = seq_df[FEATURES].values
    seq = normalize(seq)
    seq = torch.from_numpy(seq).float().to(device)
    seq = seq.unsqueeze(0)  # adding batch dim
    return seq


def preprocess_defog_seq(seq_df, device, down_hz=None):
    if down_hz is None:
        # upsampling the data from 100Hz to 128Hz
        # seq = upsample_defog(seq)
        seq_df = resample_seq_df(
            seq_df, 100, 128, with_classes=False, with_bool_cols=False
        )
    else:
        # downsample the data from 100Hz to ??Hz
        seq_df = resample_seq_df(
            seq_df, 100, down_hz, with_classes=False, with_bool_cols=False
        )

    # upsampling the data from 100Hz to 128Hz
    # seq_df = upsample_defog(seq_df, with_categorical_cols=False)

    # defog data is in g: we convert it into m/s^2
    seq_df = convert_g_to_ms2(seq_df)

    return preprocess_tdcs_seq(seq_df, device)


def predict_lbls_df(model, seq, seq_id):
    #     with autocast():  # mixed precision
    pred, h = model(seq)
    pred = torch.nn.functional.softmax(pred[0], dim=1)
    pred = pred.cpu().numpy()[:, :3]
    return pred


def resample_seq(seq_inhz, in_hz, out_hz):
    out_size = int(seq_inhz.shape[0] * (out_hz / in_hz))
    time_inhz = np.linspace(0, 1, seq_inhz.shape[0])
    time_outhz = np.linspace(0, 1, out_size)

    seq_outhz = np.zeros((out_size, seq_inhz.shape[1]))

    for i in range(seq_inhz.shape[1]):
        interp_func = interp1d(time_inhz, seq_inhz[:, i])
        seq_outhz[:, i] = interp_func(time_outhz)

    return seq_outhz


def build_res_df(preds):
    CLASSES = ["StartHesitation", "Turn", "Walking"]
    steps_ids = [(seq_id + "_" + str(i)) for i in range(preds.shape[0])]
    res_df = pd.DataFrame(data=preds, columns=CLASSES, index=steps_ids)
    return res_df

In [ ]:
MODELS_FOLDER = "models"
model_id = "best_model"  # can be changed to "simplified_model"
model_path = os.path.join(MODELS_FOLDER, model_id, "model.pth")
params_path = os.path.join(MODELS_FOLDER, model_id, "params.json")
with open(params_path, "r", encoding="utf-8") as f:
    PARAMS = json.load(f)

model = MultiResidualBiGRU(
    PARAMS["ISIZE"],
    PARAMS["HSIZE"],
    PARAMS["NC"],
    PARAMS["NL"],
)

model = model.to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

In [ ]:
TEST_ROOT_PATH = os.path.join(BASE_FOLDER, "test")
defog_TEST_PATH = os.path.join(TEST_ROOT_PATH, "defog")
tDCS_TEST_PATH = os.path.join(TEST_ROOT_PATH, "tdcsfog")

defog_test_fpaths = glob.glob(os.path.join(defog_TEST_PATH, "**"))
tdcs_test_fpaths = glob.glob(os.path.join(tDCS_TEST_PATH, "**"))

SUB_PATH = os.path.join(BASE_FOLDER, "sample_submission.csv")
sub_df = pd.read_csv(SUB_PATH)

# Id column temporarily becomes the explicit pandas index
sub_df.set_index("Id", inplace=True)

with torch.no_grad():
    for root_path in [tDCS_TEST_PATH, defog_TEST_PATH]:
        fpaths = glob.glob(os.path.join(root_path, "**"))
        for fpath in fpaths:
            # reading the sequence and its id
            seq_id, seq = read_seq(fpath)

            # preprocessing the sequence
            if root_path == defog_TEST_PATH:
                seq = preprocess_defog_seq(
                    seq, device, down_hz=PARAMS["DOWNHZ"]
                )
            else:
                seq = preprocess_tdcs_seq(seq, device, down_hz=PARAMS["DOWNHZ"])

            # using the model to predict labels
            preds = predict_lbls_df(model, seq, seq_id)

            if root_path == defog_TEST_PATH:
                in_hz = 128 if PARAMS["DOWNHZ"] is None else PARAMS["DOWNHZ"]
                preds = resample_seq(preds, in_hz, 100)
            elif PARAMS["DOWNHZ"] is not None:  # tdcs
                preds = resample_seq(preds, PARAMS["DOWNHZ"], 128)

            res_df = build_res_df(preds)

            # updating the submission dataframe with these partial results
            sub_df.update(res_df)

# making Id back to a column and saving the dataframe in a csv file
sub_df.reset_index(inplace=True)
sub_df.to_csv("submission.csv", index=False)

display(sub_df)